In [1]:
import numpy as np
import staintools
import torch
from models import StainNet, ResnetGenerator
from PIL import Image 
import matplotlib.pyplot as plt
%matplotlib inline
import os
import pickle
import pandas as pd
import tracemalloc
import pickle
import shutil
import time
import cv2

In [2]:
#load  pretrained StainNet
model_Net = StainNet().cuda()
model_Net.load_state_dict(torch.load("checkpoints/aligned_cytopathology_dataset/StainNet-3x0_best_psnr_layer3_ch32.pth"))

<All keys matched successfully>

In [3]:
def norm(image):
    image = np.array(image).astype(np.float32)
    image = image.transpose((2, 0, 1))
    image = ((image / 255) - 0.5) / 0.5
    image=image[np.newaxis, ...]
    image=torch.from_numpy(image)
    return image

def un_norm(image):
    image = image.cpu().detach().numpy()[0]
    image = ((image * 0.5 + 0.5) * 255).astype(np.uint8).transpose((1,2,0))
    return image


In [4]:
#load  pretrained StainGAN
model_GAN = ResnetGenerator(3, 3, ngf=64, norm_layer=torch.nn.InstanceNorm2d, n_blocks=9).cuda().cuda()
model_GAN.load_state_dict(torch.load("checkpoints/aligned_cytopathology_dataset/latest_net_G_A.pth"))

<All keys matched successfully>

# For all patches

In [5]:
PATH =  "/media/user/Watson/PanNuke/normalisation_study/original/patches/"
#PATH = '/media/user/Ligue_RNA_2partie/PCNSL_2022/patches_tiatoolbox/'
PATH_GAN =  "/media/user/Watson/PanNuke/Normalisation/model_GAN/"
PATH_NET =  "/media/user/Watson/PanNuke/Normalisation/model_NET/"

In [6]:
#load  pretrained StainGAN
model_GAN = ResnetGenerator(3, 3, ngf=64, norm_layer=torch.nn.InstanceNorm2d, n_blocks=9).cuda().cuda()
model_GAN.load_state_dict(torch.load("checkpoints/aligned_cytopathology_dataset/latest_net_G_A.pth"))

<All keys matched successfully>

In [7]:
organs = ['Adrenal_gland', 'Bile-duct', 'Bladder', 'Breast', 'Cervix', 'Colon', 'Esophagus', 'HeadNeck', 'Kidney', 'Liver', 'Lung', 'Ovarian', 'Pancreatic', 'Prostate', 'Skin', 'Stomach', 'Testis', 'Thyroid', 'Uterus']

110

In [1]:
for organ in organs :

    if not os.path.exists(PATH_GAN+organ) :
        os.mkdir(PATH_GAN+organ)
        for i in os.listdir(PATH+organ):
            img_source = Image.open(PATH+organ+'/'+i)
            image_gan=model_GAN(norm(img_source).cuda())
            image_gan=un_norm(image_gan)
            cv2.imwrite(PATH_GAN+organ+'/'+i,image_gan)


In [2]:
for organ in organs :
    if not os.path.exists(PATH_NET+organ) :
        os.mkdir(PATH_NET+organ)
        for i in os.listdir(PATH+organ):
            img_source = Image.open(PATH+organ+'/'+i)
            image_gan=model_Net(norm(img_source).cuda())
            image_gan=un_norm(image_gan)
            cv2.imwrite(PATH_NET+organ+'/'+i,image_gan)
